In [1]:
using Distributions, TimeIt, ProgressMeter, PyPlot, JLD
include("/home/postdoc/dsen/Desktop/G-ZZ/jl_files/zz_samplers.jl")
include("temp.jl")

run_sampler (generic function with 4 methods)

In [2]:
d, Nobs = 51, 100
pX = 5e-1
pξ = 1e-1
X = sprandn(d, Nobs, pX)
X[1,:] = 1
ξ_true = sprandn(d, pξ)
y = Float64.([rand(Binomial(1, 1/(1+exp(-ξ_true'X[:,j])))) for j in 1:Nobs])
σ02 = 1
my_pr = SS_prior(d, σ02)
my_ll = ll_logistic_sp(X,y)
my_model = model(my_ll, my_pr);

### GZZ:

In [3]:
# Sub-sampling without control variates and with weights:
mb_size = 1
max_attempts = 10^5
lambda = 5e1
Print = true
prob_het = 0.98
adapt_speed = "none";

In [4]:
gs = Array{mbsampler}(d)
gs[1] = umbsampler(0, Nobs, mb_size)

for i in 2:d
    weights_het = abs.(my_model.ll.X[i,:])./sum(abs.(my_model.ll.X[i,:]))
    if length(my_model.ll.X[i,:].nzind) < length(my_model.ll.X[i,:]) 
        gs[i] = spwumbsampler(Nobs, mb_size, weights_het, prob_het)
    else 
        gs[i] = wumbsampler(Nobs, mb_size, weights_het)
    end
end
gs_list = mbsampler_list(d,gs);

# Define output scheduler etc:
A_xi = eye(d)
#A_hyp = eye(hyperparam_size(my_model.pr))
A_hyp = ones(1,hyperparam_size(my_model.pr))/hyperparam_size(my_model.pr)

opf = projopf(A_xi, A_hyp, 10^3)
opt = maxa_opt(max_attempts)
outp = outputscheduler(opf,opt)
bb = linear_bound(my_model.ll, my_model.pr, gs_list)
mstate = zz_state(d)
update_bound(bb, my_model.ll, my_model.pr, gs_list, mstate)

# Define block Gibbs sampler:
L = 1
my_zz_sampler = zz_sampler(0, gs_list, bb, L, adapt_speed)
hyper_sampler = block_gibbs_sampler(lambda)
blocksampler = Array{msampler}(2)
blocksampler[1] = my_zz_sampler
blocksampler[2] = hyper_sampler

# Run sampler:
ZZ_block_sample(my_model, outp, blocksampler, mstate, Print);

10 percent attempts in 0.08 min; zz bounces = 1710, hyp bounces = 218, total time of process = 4.453 
20 percent attempts in 0.12 min; zz bounces = 3037, hyp bounces = 442, total time of process = 8.907 
30 percent attempts in 0.16 min; zz bounces = 4233, hyp bounces = 694, total time of process = 13.433 
40 percent attempts in 0.20 min; zz bounces = 5441, hyp bounces = 916, total time of process = 17.940 
50 percent attempts in 0.24 min; zz bounces = 6690, hyp bounces = 1146, total time of process = 22.457 
60 percent attempts in 0.29 min; zz bounces = 7786, hyp bounces = 1360, total time of process = 26.988 
70 percent attempts in 0.33 min; zz bounces = 9050, hyp bounces = 1587, total time of process = 31.547 
80 percent attempts in 0.37 min; zz bounces = 10303, hyp bounces = 1810, total time of process = 36.028 
90 percent attempts in 0.42 min; zz bounces = 11646, hyp bounces = 2027, total time of process = 40.417 
100 percent attempts in 0.46 min; zz bounces = 13038, hyp bounces = 